# 🔬 Cellpose Introduction - Cell Segmentation Tutorial

Welcome! This notebook will guide you through using Cellpose for cell segmentation with no coding experience required.

## What is Cellpose?
Cellpose is an AI-powered tool that automatically identifies and segments cells in microscopy images. It can distinguish between different cells and create masks showing where each cell is located.

## How to use this notebook:
1. Click on each cell and press **Shift + Enter** to run it
2. Or click **Run All** in the toolbar above
3. Follow the instructions in each section

Let's get started! 🚀

## Step 1: Setup and Installation Check

In [ ]:
# Import required libraries
print("🔧 Setting up Cellpose environment...")

import warnings
warnings.filterwarnings('ignore')

try:
    # Core libraries
    import numpy as np
    import matplotlib.pyplot as plt
    from matplotlib.patches import Rectangle
    import os
    
    # Cellpose
    from cellpose import models, io, plot, utils
    
    # Widgets for interaction
    import ipywidgets as widgets
    from IPython.display import display, clear_output, HTML
    
    print("✅ All libraries imported successfully!")
    print(f"✅ Cellpose version: {models.__version__ if hasattr(models, '__version__') else 'installed'}")
    
    # Create results directory
    os.makedirs('../results', exist_ok=True)
    
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Please restart the kernel and try again.")
    
print("\n🎉 Ready to segment cells!")

## Step 2: Load Sample Image

Let's start with a sample image to see how Cellpose works:

In [ ]:
# Create a sample image if none exists
sample_dir = '../sample_data'
os.makedirs(sample_dir, exist_ok=True)

# Check for existing sample images
sample_files = []
if os.path.exists(sample_dir):
    for ext in ['*.png', '*.jpg', '*.jpeg', '*.tif', '*.tiff']:
        import glob
        sample_files.extend(glob.glob(os.path.join(sample_dir, ext)))

if sample_files:
    print(f"📁 Found {len(sample_files)} sample images:")
    for f in sample_files:
        print(f"  - {os.path.basename(f)}")
    
    # Use the first sample image
    sample_image_path = sample_files[0]
    print(f"\n🖼️ Using: {os.path.basename(sample_image_path)}")
else:
    # Create a synthetic sample image
    print("📸 Creating a sample image for demonstration...")
    
    # Generate synthetic cell-like image
    np.random.seed(42)
    img_size = 256
    img = np.zeros((img_size, img_size))
    
    # Add some circular "cells"
    from scipy.ndimage import gaussian_filter
    
    for i in range(8):
        # Random position and size
        x = np.random.randint(30, img_size-30)
        y = np.random.randint(30, img_size-30)
        r = np.random.randint(15, 25)
        
        # Create circular mask
        Y, X = np.ogrid[:img_size, :img_size]
        mask = (X - x)**2 + (Y - y)**2 <= r**2
        img[mask] = 200 + np.random.randint(-50, 50)
    
    # Add some noise and blur
    img = gaussian_filter(img, sigma=1.5)
    img = img + np.random.normal(0, 10, img.shape)
    img = np.clip(img, 0, 255).astype(np.uint8)
    
    # Save sample image
    sample_image_path = os.path.join(sample_dir, 'synthetic_cells.png')
    from PIL import Image
    Image.fromarray(img).save(sample_image_path)
    print(f"✅ Created synthetic sample: {sample_image_path}")

# Load and display the image
try:
    img = io.imread(sample_image_path)
    print(f"📊 Image shape: {img.shape}")
    print(f"📊 Image type: {img.dtype}")
    
    # Display the image
    plt.figure(figsize=(8, 6))
    if len(img.shape) == 2:
        plt.imshow(img, cmap='gray')
    else:
        plt.imshow(img)
    plt.title(f'Sample Image: {os.path.basename(sample_image_path)}')
    plt.axis('off')
    plt.show()
    
except Exception as e:
    print(f"❌ Error loading image: {e}")

## Step 3: Interactive Cellpose Segmentation

Now let's create an easy-to-use interface for running Cellpose:

In [ ]:
# Interactive Cellpose widget

def run_cellpose_segmentation(model_type, diameter, image_path):
    """Run Cellpose segmentation with given parameters"""
    
    try:
        print(f"🔬 Loading {model_type} model...")
        model = models.Cellpose(model_type=model_type)
        
        print(f"📷 Processing image: {os.path.basename(image_path)}")
        img = io.imread(image_path)
        
        print("🎯 Running segmentation...")
        # Set channels - [0,0] for grayscale, [2,3] for RGB with nucleus in blue
        channels = [0, 0] if len(img.shape) == 2 else [0, 0]
        
        masks, flows, styles, diams = model.eval(
            img, 
            diameter=diameter if diameter > 0 else None,
            channels=channels
        )
        
        # Count cells
        n_cells = len(np.unique(masks)) - 1  # subtract background
        
        # Create visualization
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        # Original image
        if len(img.shape) == 2:
            axes[0].imshow(img, cmap='gray')
        else:
            axes[0].imshow(img)
        axes[0].set_title('Original Image')
        axes[0].axis('off')
        
        # Segmentation masks
        axes[1].imshow(masks, cmap='tab20')
        axes[1].set_title(f'Segmentation Masks\n({n_cells} cells found)')
        axes[1].axis('off')
        
        # Overlay
        overlay = plot.mask_overlay(img, masks, colors=None)
        axes[2].imshow(overlay)
        axes[2].set_title('Overlay')
        axes[2].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        # Save results
        result_dir = '../results'
        mask_path = os.path.join(result_dir, f'masks_{os.path.basename(image_path)}.png')
        overlay_path = os.path.join(result_dir, f'overlay_{os.path.basename(image_path)}.png')
        
        # Save mask
        io.imsave(mask_path, masks.astype(np.uint16))
        
        # Save overlay
        plt.figure(figsize=(10, 10))
        plt.imshow(overlay)
        plt.axis('off')
        plt.savefig(overlay_path, bbox_inches='tight', dpi=150)
        plt.close()
        
        print(f"\n✅ Segmentation complete!")
        print(f"📊 Found {n_cells} cells")
        print(f"💾 Results saved to: {result_dir}")
        print(f"   - Masks: {os.path.basename(mask_path)}")
        print(f"   - Overlay: {os.path.basename(overlay_path)}")
        
        return masks, img, n_cells
        
    except Exception as e:
        print(f"❌ Error during segmentation: {e}")
        print("\nTroubleshooting tips:")
        print("- Try a different model type")
        print("- Adjust the diameter setting")
        print("- Check your image format")
        return None, None, 0

print("🛠️ Segmentation function ready!")

## Step 4: Easy Controls

Use these simple controls to adjust Cellpose settings:

In [ ]:
# Create interactive controls

# Model selection
model_dropdown = widgets.Dropdown(
    options=[
        ('Cells (general)', 'cyto'),
        ('Nuclei', 'nuclei'),
        ('Cells (cytoplasm2)', 'cyto2')
    ],
    value='cyto',
    description='Model Type:',
    style={'description_width': 'initial'}
)

# Diameter setting
diameter_slider = widgets.IntSlider(
    value=30,
    min=0,
    max=100,
    step=5,
    description='Cell Diameter:',
    style={'description_width': 'initial'},
    tooltip='Average cell diameter in pixels (0 for auto-detection)'
)

# Run button
run_button = widgets.Button(
    description='🎯 Run Cellpose',
    button_style='success',
    layout=widgets.Layout(width='200px', height='50px')
)

# Output area
output_area = widgets.Output()

def on_run_button_click(button):
    """Handle run button click"""
    with output_area:
        clear_output(wait=True)
        print("🚀 Starting segmentation...")
        
        # Run segmentation
        masks, img, n_cells = run_cellpose_segmentation(
            model_type=model_dropdown.value,
            diameter=diameter_slider.value,
            image_path=sample_image_path
        )

# Connect button to function
run_button.on_click(on_run_button_click)

# Display interface
print("🎛️ Cellpose Control Panel:")
print("\n📖 Instructions:")
print("1. Choose a model type (cyto for general cells, nuclei for nucleus-only)")
print("2. Set cell diameter (30 pixels is often good, 0 for auto-detection)")
print("3. Click 'Run Cellpose' to segment the image")
print("\n⚙️ Settings:")

display(widgets.VBox([
    model_dropdown,
    diameter_slider,
    run_button
]))

display(output_area)

## Step 5: Upload Your Own Images (Optional)

Want to try Cellpose on your own images? Use the file upload widget below:

In [ ]:
# File upload widget
from ipywidgets import FileUpload, Button, Output, VBox, HBox

# File uploader
uploader = FileUpload(
    accept='.png,.jpg,.jpeg,.tif,.tiff',
    multiple=False,
    description='Upload Image'
)

# Process uploaded file button
process_upload_button = Button(
    description='📸 Process Uploaded Image',
    button_style='info',
    disabled=True
)

upload_output = Output()

def on_upload_change(change):
    """Handle file upload"""
    if len(uploader.value) > 0:
        process_upload_button.disabled = False
        with upload_output:
            clear_output()
            print(f"📁 File uploaded: {list(uploader.value.keys())[0]}")
            print("Click 'Process Uploaded Image' to run Cellpose on your image.")

def process_uploaded_image(button):
    """Process the uploaded image"""
    global sample_image_path  # Update the global path
    
    with upload_output:
        clear_output(wait=True)
        
        try:
            # Get uploaded file
            filename = list(uploader.value.keys())[0]
            content = uploader.value[filename]['content']
            
            # Save to sample_data directory
            uploaded_path = os.path.join(sample_dir, f"uploaded_{filename}")
            with open(uploaded_path, 'wb') as f:
                f.write(content)
            
            print(f"✅ Saved uploaded image: {uploaded_path}")
            
            # Update sample image path
            sample_image_path = uploaded_path
            
            # Display the uploaded image
            img = io.imread(uploaded_path)
            plt.figure(figsize=(8, 6))
            if len(img.shape) == 2:
                plt.imshow(img, cmap='gray')
            else:
                plt.imshow(img)
            plt.title(f'Uploaded Image: {filename}')
            plt.axis('off')
            plt.show()
            
            print("\n🎯 Now use the controls above to run Cellpose on your uploaded image!")
            
        except Exception as e:
            print(f"❌ Error processing upload: {e}")

# Connect handlers
uploader.observe(on_upload_change, names='value')
process_upload_button.on_click(process_uploaded_image)

print("📤 Upload Your Own Image:")
print("Supported formats: PNG, JPG, TIFF")

display(VBox([
    uploader,
    process_upload_button,
    upload_output
]))

## Step 6: Understanding the Results

Let's learn how to interpret Cellpose results:

In [ ]:
print("📊 Understanding Cellpose Results:")
print("\n🎨 What you see in the results:")
print("1. Original Image - Your input image")
print("2. Segmentation Masks - Each cell gets a unique color/number")
print("3. Overlay - Masks overlaid on the original image")

print("\n🔢 Mask Values:")
print("- Background pixels = 0")
print("- First cell = 1")
print("- Second cell = 2")
print("- And so on...")

print("\n💾 Saved Files:")
print("- masks_*.png: Segmentation masks (16-bit PNG)")
print("- overlay_*.png: Visual overlay for presentations")

print("\n⚙️ Parameter Tips:")
print("Model Types:")
print("  • 'cyto' - General purpose, works for most cell types")
print("  • 'nuclei' - Specifically for nucleus segmentation")
print("  • 'cyto2' - Improved general model")

print("\nDiameter Settings:")
print("  • 30 pixels: Good starting point for most images")
print("  • 0: Auto-detection (Cellpose estimates size)")
print("  • Smaller values: For small cells/high magnification")
print("  • Larger values: For big cells/low magnification")

print("\n🎯 Next Steps:")
print("- Try different model types if results aren't good")
print("- Adjust diameter based on your cell size")
print("- Check the results folder for saved files")
print("- Use the saved masks in ImageJ, Python, or other analysis software")

## Step 7: Quick Analysis (Bonus)

Let's add some basic analysis of your segmentation results:

In [ ]:
def analyze_segmentation_results():
    """Analyze the most recent segmentation results"""
    
    # Look for the most recent mask file
    result_dir = '../results'
    if not os.path.exists(result_dir):
        print("❌ No results found. Run segmentation first!")
        return
    
    mask_files = [f for f in os.listdir(result_dir) if f.startswith('masks_') and f.endswith('.png')]
    
    if not mask_files:
        print("❌ No mask files found. Run segmentation first!")
        return
    
    # Use the most recent mask file
    latest_mask_file = sorted(mask_files)[-1]
    mask_path = os.path.join(result_dir, latest_mask_file)
    
    try:
        # Load masks
        masks = io.imread(mask_path)
        
        # Get unique cell IDs (excluding background)
        cell_ids = np.unique(masks)[1:]  # Skip 0 (background)
        n_cells = len(cell_ids)
        
        if n_cells == 0:
            print("❌ No cells found in the segmentation.")
            return
        
        # Calculate cell areas
        cell_areas = []
        for cell_id in cell_ids:
            area = np.sum(masks == cell_id)
            cell_areas.append(area)
        
        cell_areas = np.array(cell_areas)
        
        # Display analysis
        print(f"📊 Analysis Results for: {latest_mask_file}")
        print(f"\n🔢 Cell Count: {n_cells}")
        print(f"\n📏 Cell Areas (in pixels):")
        print(f"   Average: {np.mean(cell_areas):.1f} pixels")
        print(f"   Median: {np.median(cell_areas):.1f} pixels")
        print(f"   Range: {np.min(cell_areas)} - {np.max(cell_areas)} pixels")
        print(f"   Std Dev: {np.std(cell_areas):.1f} pixels")
        
        # Create histogram
        plt.figure(figsize=(10, 4))
        
        plt.subplot(1, 2, 1)
        plt.hist(cell_areas, bins=min(20, n_cells), alpha=0.7, color='skyblue', edgecolor='black')
        plt.xlabel('Cell Area (pixels)')
        plt.ylabel('Number of Cells')
        plt.title('Distribution of Cell Areas')
        plt.grid(True, alpha=0.3)
        
        # Show some individual cells
        plt.subplot(1, 2, 2)
        plt.imshow(masks, cmap='tab20')
        plt.title(f'Cell Segmentation ({n_cells} cells)')
        plt.axis('off')
        
        # Add cell numbers to a few cells
        for i, cell_id in enumerate(cell_ids[:10]):  # Show first 10 cell numbers
            # Find center of mass for this cell
            y_coords, x_coords = np.where(masks == cell_id)
            if len(y_coords) > 0:
                center_y = np.mean(y_coords)
                center_x = np.mean(x_coords)
                plt.text(center_x, center_y, str(cell_id), 
                        color='white', fontweight='bold', 
                        ha='center', va='center', fontsize=8)
        
        plt.tight_layout()
        plt.show()
        
        # Summary stats
        print(f"\n📈 Summary:")
        if np.std(cell_areas) / np.mean(cell_areas) < 0.3:
            print("   ✅ Cells are fairly uniform in size")
        else:
            print("   ⚠️ High variability in cell sizes")
        
        if n_cells > 50:
            print("   📊 Large sample size - good for statistical analysis")
        elif n_cells > 10:
            print("   📊 Moderate sample size")
        else:
            print("   📊 Small sample size")
            
    except Exception as e:
        print(f"❌ Error during analysis: {e}")

# Button to run analysis
analyze_button = widgets.Button(
    description='📊 Analyze Results',
    button_style='info',
    layout=widgets.Layout(width='200px', height='40px')
)

analyze_output = widgets.Output()

def on_analyze_click(button):
    with analyze_output:
        clear_output(wait=True)
        analyze_segmentation_results()

analyze_button.on_click(on_analyze_click)

print("🔍 Want to analyze your segmentation results?")
print("Click the button below after running a segmentation:")

display(analyze_button)
display(analyze_output)

## 🎉 Congratulations!

You've successfully learned how to use Cellpose for cell segmentation! 

### What you accomplished:
- ✅ Set up Cellpose in your browser
- ✅ Ran cell segmentation on sample images
- ✅ Learned to adjust parameters
- ✅ Uploaded and processed your own images
- ✅ Analyzed segmentation results

### Next steps:
1. **Download your results** from the `results` folder
2. **Try different images** with various cell types
3. **Experiment with parameters** to optimize results
4. **Use results in other software** like ImageJ, Python, or R

### Need help?
- Check the [Cellpose documentation](https://cellpose.readthedocs.io/)
- Try different model types for your specific images
- Adjust the diameter parameter based on your cell size

### Pro tips:
- 🎯 The 'cyto' model works well for most cell types
- 📏 Diameter setting is crucial - measure a few cells to get the right value
- 🖼️ High contrast images work best
- 💾 Save your parameters when you get good results

Happy cell segmenting! 🔬✨